In [1]:
# Imports

import os
import pickle
import datetime

import numpy as np
import pandas as pd
import spacy

from datetime import datetime
from spacy.tokenizer import Tokenizer
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from category_encoders import OneHotEncoder

In [2]:
# Loading .csv

df = pd.read_csv("ks-projects-201801.csv")

In [3]:
# Dropping features, filtering down to only useful observations

df = df.drop(["pledged", "backers", "usd pledged", "usd_pledged_real", "usd_goal_real"], axis=1)

mask = df['currency'] == 'USD'
df = df[mask]
mask = df['country'] == 'US'
df = df[mask]

df = df[(df['state'] == 'successful') | (df['state'] == 'failed')]

In [4]:
# Campaign length feature engineered

df['deadline']= pd.to_datetime(df['deadline'])
df['launched'] = pd.to_datetime(df['launched'])
df['campaign length'] = df['deadline'] - df['launched']

df['campaign length'] = df['campaign length'].astype(str)
df['campaign length'] = df['campaign length'].apply(lambda x: x[:2])

In [5]:
# Dropping other features

df = df.drop(['deadline', 'launched', 'country', 'ID', 'currency', 'main_category'], axis=1)

In [6]:
# Using small sample of dataframe to reduce computational time

df = df.sample(1000)

In [7]:
# NLP for vectorization, pipeline using OHE and GBC predictor

nlp = spacy.load("en_core_web_md")
gbc = GradientBoostingClassifier(loss='deviance')
ohe = OneHotEncoder()
pipe = Pipeline([
    ('encoder', ohe),
    ('clf', gbc)
    ])

In [8]:
# Vectorization of text feature

df2 = pd.DataFrame([nlp(str(d)).vector for d in df['name']])

In [9]:
# Adding features onto vector dataframe

df.index = df2.index
df2['category'] = df['category']
df2['goal'] = df['goal']
df2['campaign length'] = df['campaign length'].astype(int)

In [10]:
# Feature matrix and target

X = df2
y = df['state']

In [11]:
# Train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=1, 
                                                    shuffle=True,
                                                    stratify=y)

In [12]:
# Fitting model

pipe.fit(X,y)

/Users/micahswain/.local/share/virtualenvs/kickstarter_success-cyjBXNL6/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('encoder', OneHotEncoder(cols=['category'])),
                ('clf', GradientBoostingClassifier())])

In [13]:
# Pickling model

gbnlp = open('gbnlp_pickle', 'wb') 
pickle.dump(pipe, gbnlp)

In [14]:
# Loading pickled model

loaded_model = pickle.load(open('gbnlp_pickle', 'rb'))

In [15]:
type(df2.head(1))

pandas.core.frame.DataFrame

In [16]:
loaded_model.predict(X.head(1))

array(['failed'], dtype=object)

In [17]:
goal = 500
length=30
description='Do it'
category='Game'

In [18]:
desc_vectorized = pd.DataFrame(nlp(description).vector)

In [19]:
X_new = desc_vectorized.T
X_new['category'] = category
X_new['goal'] = goal
X_new['campaign length'] = length

In [20]:
X_new

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,category,goal,campaign length
0,-0.006584,0.142384,-0.325879,-0.163085,0.007788,0.021567,-0.104288,-0.079318,-0.15804,2.58415,...,-0.113279,0.101291,-0.08756,-0.18719,-0.1184,0.357899,0.1733,Game,500,30


In [21]:
y_pred = loaded_model.predict(X_new)

In [22]:
loaded_model.predict_proba(X_new)

array([[0.84285513, 0.15714487]])

In [30]:
set(df2['category'])

{'Academic',
 'Accessories',
 'Action',
 'Animation',
 'Anthologies',
 'Apparel',
 'Apps',
 'Architecture',
 'Art',
 'Art Books',
 'Audio',
 'Bacon',
 'Calendars',
 'Candles',
 "Children's Books",
 'Civic Design',
 'Classical Music',
 'Comedy',
 'Comic Books',
 'Comics',
 'Community Gardens',
 'Conceptual Art',
 'Cookbooks',
 'Country & Folk',
 'Crafts',
 'DIY',
 'DIY Electronics',
 'Dance',
 'Design',
 'Digital Art',
 'Documentary',
 'Drama',
 'Drinks',
 'Electronic Music',
 'Events',
 'Experimental',
 'Fabrication Tools',
 'Faith',
 'Family',
 'Farms',
 'Fashion',
 'Festivals',
 'Fiction',
 'Film & Video',
 'Fine Art',
 'Flight',
 'Food',
 'Food Trucks',
 'Footwear',
 'Gadgets',
 'Games',
 'Gaming Hardware',
 'Glass',
 'Graphic Design',
 'Graphic Novels',
 'Hardware',
 'Hip-Hop',
 'Horror',
 'Illustration',
 'Immersive',
 'Indie Rock',
 'Installations',
 'Interactive Design',
 'Jazz',
 'Jewelry',
 'Journalism',
 'Kids',
 'Live Games',
 'Metal',
 'Mixed Media',
 'Mobile Games',
 'Musi